# **Sistema de recomendação baseado em conteúdo**
- Implementaremos um sistema de recomendação baseado em conteúdo. Os sistemas de recomendação baseados em conteúdo sugerem itens aos usuários com base nas características dos itens e em um perfil de preferências do usuário.

## **1. Introdução**
- **O que é um sistema de recomendação baseado em conteúdo?**
    - Um sistema de recomendação baseado em conteúdo recomenda itens aos usuários com base no conteúdo ou nas características dos itens. Este tipo de sistema de recomendação concentra-se na compreensão das propriedades dos itens e no aprendizado das preferências do usuário a partir dos itens com os quais eles interagiram no passado.


- **Como funciona?**
    - O princípio de funcionamento de um sistema de recomendação baseado em conteúdo pode ser resumido em alguns passos:
        1. **Extração de recursos**: Extraia recursos relevantes dos itens. Por exemplo, em um sistema de recomendação de livros, os recursos podem incluir título, autor e categoria

        2. **Perfil de usuário**: crie um perfil de usuário com base em suas interações com os itens. Este perfil é essencialmente um resumo das características dos itens que o usuário gostou ou com os quais interagiu no passado.

        3. **Recomendação**: Calcule a semelhança entre o perfil do usuário e as características de cada item. São recomendados os itens que mais se assemelham ao perfil do usuário.

## 2. **Preparação de dados**
**Conjunto de dados**
   - Usaremos um conjunto de dados contendo informações dos livros, incluindo títulos, autores e categorias.

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_csv('/content/livros_exportados.csv')

In [ ]:
df.head(20)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5014 entries, 0 to 5013
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ISBN            5014 non-null   int64 
 1   Title           5014 non-null   object
 2   Authors         4940 non-null   object
 3   Categories      4290 non-null   object
 4   Description     5014 non-null   object
 5   Published Date  5014 non-null   object
 6   Language        5014 non-null   object
 7   Num Pages       5014 non-null   int64 
dtypes: int64(2), object(6)
memory usage: 313.5+ KB


** Extração de recursos **
- Extrairemos características relevantes do conjunto de dados, como títulos de livros e autores.

In [5]:
# selecionando feactures importantes
selected_features = ['Title','Authors','Categories','Language']
print(selected_features)

['Title', 'Authors', 'Categories', 'Language']


**Pré-processamento de dados**
- Antes de construir o sistema de recomendação, precisamos pré-processar os dados. Isso pode incluir limpeza de texto, tratamento de valores ausentes e tokenização.

In [6]:
# Substituindo os valores nulos por string nula
for feature in selected_features:
    df[feature] = df[feature].fillna('')

In [7]:
# combinando todos os 4 recursos selecionados
combined_features = df['Title'] + ' ' + df['Categories'] + ' ' + df['Authors'] + ' ' + f"{df['Language']}"
combined_features

,0
0,Armadilha aérea Adventure stories Michael Cric...
1,Gostos adquiridos Affluent consumers Peter May...
2,A campanha Carlos Fuentes 0 pt-BR\n1 ...
3,Lembre-se de mim Cape Cod (Mass.) Mary Higgins...
4,O cliente Mafia John Grisham 0 pt-BR\n1 ...
...,...
5009,The Pizza Monster Juvenile Fiction Marjorie We...
5010,Hoot Bullying Carl Hiaasen 0 pt-BR\n1 ...
5011,War Comes to Willy Freeman Juvenile Fiction Ja...
5012,A Wrinkle in Time Juvenile Fiction Madeleine L...


## 3. **Construindo o sistema de recomendação baseado em conteúdo**
**Vetorização TF-IDF**
- Usamos a vetorização TF-IDF (Term Frequency-Inverse Document Frequency) para converter recursos de texto (descrições) em vetores numéricos.
- O TF-IDF dá mais peso aos termos que são importantes num documento específico e menos peso aos termos comuns.

In [8]:
# convertendo os dados de texto em vetores de recursos
vectorizer = TfidfVectorizer()

feature_vectors = vectorizer.fit_transform(combined_features)

In [9]:
print(feature_vectors)

  (0, 611)	0.44242042550446004
  (0, 783)	0.44242042550446004
  (0, 193)	0.3351581020482882
  (0, 8992)	0.26717186009924293
  (0, 6275)	0.27912126650006935
  (0, 2236)	0.36175373730071786
  (0, 7716)	0.25060515718106385
  (0, 1234)	0.25060515718106385
  (0, 83)	0.05012103143621277
  (0, 3091)	0.25060515718106385
  (0, 84)	0.05012103143621277
  (0, 85)	0.05012103143621277
  (0, 86)	0.05012103143621277
  (0, 87)	0.05012103143621277
  (0, 6600)	0.05012103143621277
  (0, 5374)	0.05012103143621277
  (0, 5476)	0.05012103143621277
  (0, 88)	0.05012103143621277
  (0, 2865)	0.05012103143621277
  (0, 6862)	0.05012103143621277
  (1, 7716)	0.2232309753252258
  (1, 1234)	0.2232309753252258
  (1, 83)	0.044646195065045156
  (1, 3091)	0.2232309753252258
  (1, 84)	0.044646195065045156
  :	:
  (5013, 84)	0.029979938936169948
  (5013, 85)	0.029979938936169948
  (5013, 86)	0.029979938936169948
  (5013, 87)	0.029979938936169948
  (5013, 6600)	0.029979938936169948
  (5013, 5374)	0.029979938936169948
  (5013

**Semelhança de cosseno**
- Calculamos a similaridade de cosseno entre os vetores de itens TF-IDF. A similaridade de cosseno mede o cosseno do ângulo entre dois vetores diferentes de zero e é usada para determinar o quão semelhantes dois itens são com base em seus vetores de características.

In [10]:
# obtendo as pontuações de similaridade usando similaridade de cosseno
similarity = cosine_similarity(feature_vectors, feature_vectors)

In [11]:
print(similarity)

[[1.         0.19244335 0.26141389 ... 0.18651879 0.22711307 0.12922579]
 [0.19244335 1.         0.23285904 ... 0.16614491 0.20230498 0.11511016]
 [0.26141389 0.23285904 1.         ... 0.22569024 0.27480987 0.15636495]
 ...
 [0.18651879 0.16614491 0.22569024 ... 1.         0.2375304  0.11156638]
 [0.22711307 0.20230498 0.27480987 ... 0.2375304  1.         0.13584788]
 [0.12922579 0.11511016 0.15636495 ... 0.11156638 0.13584788 1.        ]]


**Teste**

In [12]:
# criando uma lista com todos os nomes de livros fornecidos no conjunto de dados

list_of_all_titles = df['Title'].tolist()
print(list_of_all_titles)

['Armadilha aérea', 'Gostos adquiridos', 'A campanha', 'Lembre-se de mim', 'O cliente', 'O 13ð jurado', 'Os numeros da natureza', 'Flores raras e banalíssimas', 'Sonhos de uma teoria final a busca das leis fundamentais da natureza', 'O quark e o jaguar', 'A lei da fazenda', 'Limite', 'A espinha dorsal da memória', 'A conexão Bellarosa', 'As vidas de Chico Xavier', 'Hildegard de Bingen', 'Shakespeare apaixonado', 'A mulher que eles chamavam fatal', 'A grande fala do índio guarani e A catedral de Colônia', 'Pavarotti, meu mundo', 'Arthur Bispo do Rosario', 'A Escolha da Dra. Cole', 'O cavaleiro do medo', 'Lasher', 'Taltos', 'Reorganize sua bagagem', 'A lua dos namorados', 'Keith Jarrett no blue note', 'Jesuíno, o profeta', 'PRONTO', 'Palimpsesto', 'O rio que saía do Éden', 'As valkírias', 'O diamente de Jerusalém', 'Forrest Gump : o contador de historias', 'O servo dos ossos', 'O bico do tentilhão', 'Tempo de violênica', 'A câmara', 'Afirma Pereira', 'Zona quente', 'Amsterdam', 'O retorn

In [25]:
book_name = input(' Entre com o nome do livro ') # input: sherlock holmes

 Entre com o nome do livro  sherlock holmes


In [26]:
# encontrando a correspondência aproximada para o nome do livro fornecido pelo usuário
find_close_match = difflib.get_close_matches(book_name, list_of_all_titles)
print(find_close_match)

['A volta de Sherlock Holmes', 'A volta de Sherlock Holmes', 'The Memoirs of Sherlock Holmes']


In [27]:
#encontrando o índice do livro com título
close_match = find_close_match[0]
index_of_the_book = df[df.Title == close_match].index[0]

In [28]:
# obtendo uma lista de livros semelhantes
similarity_score = list(enumerate(similarity[index_of_the_book]))
print(similarity_score)

[(0, 0.2224460000448173), (1, 0.19814770815479402), (2, 0.2691626555936365), (3, 0.19447802488054008), (4, 0.2528762654933653), (5, 0.2399918457075616), (6, 0.23258310265038512), (7, 0.21180211043726524), (8, 0.18137484814482518), (9, 0.20764705930750268), (10, 0.19698658139373956), (11, 0.1937355011642839), (12, 0.19291411778848017), (13, 0.1776880070279907), (14, 0.21560762217839236), (15, 0.24467865217080415), (16, 0.2167346827867776), (17, 0.15960880420450813), (18, 0.19141783493529207), (19, 0.1392525407244331), (20, 0.24355041122802376), (21, 0.2273834932646173), (22, 0.24968317115362074), (23, 0.22439990472982047), (24, 0.2224203832918452), (25, 0.19941281714690637), (26, 0.21224713902966547), (27, 0.18786865144205847), (28, 0.20451399168008066), (29, 0.2585899539224943), (30, 0.2390696079521586), (31, 0.21526030372382376), (32, 0.2582205650006118), (33, 0.2666355308171051), (34, 0.21508975058258137), (35, 0.2200088924293026), (36, 0.22760204634354006), (37, 0.2612402949517558),

In [29]:
# classificando os livros com base na pontuação de similaridade
sorted_similar_books = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_books)

[(2084, 0.9999999999999996), (2212, 0.9999999999999996), (2219, 0.8185198225035556), (2820, 0.815549046580893), (2083, 0.8125853973882683), (4679, 0.744560518730974), (1546, 0.6869647708093907), (2117, 0.675859681948776), (4942, 0.6548966544574981), (2186, 0.5773998809187411), (2183, 0.5671198113618376), (2184, 0.5532596067684766), (1542, 0.5358584555772206), (2185, 0.46871515482297854), (1032, 0.4641681009433651), (4683, 0.4608445440516581), (1801, 0.43234864391449346), (3401, 0.4204810426417347), (3378, 0.38330121456072563), (3393, 0.38330121456072563), (1392, 0.38262248699664414), (1980, 0.37759448629905334), (1021, 0.3725644952322341), (4651, 0.3697027144796743), (3398, 0.3691664600179183), (3646, 0.366481457338347), (622, 0.365348532153657), (4751, 0.35429201975506674), (3394, 0.35291275939143985), (4908, 0.35291275939143985), (2292, 0.3514114476472456), (3459, 0.34960844922890205), (151, 0.3466514733081309), (1166, 0.3466514733081309), (1824, 0.3466514733081309), (3988, 0.3439901

In [30]:
top_sim = sorted_similar_books[:5]
top_sim

[(2084, 0.9999999999999996),
 (2212, 0.9999999999999996),
 (2219, 0.8185198225035556),
 (2820, 0.815549046580893),
 (2083, 0.8125853973882683)]

In [33]:
# imprime o nome de livros semelhantes com base no índice
i = 0

for book in sorted_similar_books:
    index = book[0]
    title_from_index = df[df.index==index]['Title'].values[0]
    if (i < 6):
        print(i, '-', title_from_index)
        i += 1

0 - A volta de Sherlock Holmes
1 - A volta de Sherlock Holmes
2 - Os arquivos de Sherlock Holmes
3 - The Memoirs of Sherlock Holmes
4 - The adventures of Sherlock Holmes
5 - New Annotated Sherlock Holmes


## **Sistema de recomendação completo**

In [22]:
#A História da Ciência para quem tem pressa

In [23]:
book_name = input('Livro de entrada : ')

list_of_all_titles = df['Title'].tolist()

find_close_match = difflib.get_close_matches(book_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_book = df[df.Title == close_match].index[0]

similarity_score = list(enumerate(similarity[index_of_the_book]))

sorted_similar_books = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Livros sugeridos : \n')

i = 1

for book in sorted_similar_books:
    index = book[0]
    title_from_index = df[df.index==index]['Title'].values[0]
    if (i < 30):
        print(i, '.',title_from_index)
        i+=1

Livro de entrada : A História da Ciência para quem tem pressa
Livros sugeridos : 

1 . A História da Ciência para quem tem pressa
2 . A História do Século 20 para quem tem pressa
3 . A história da ciência para quem tem pressa
4 . A História da Mitologia para quem tem pressa
5 . A História da Astrologia para quem tem pressa
6 . A História do Universo para quem tem pressa
7 . A História do Futebol para quem tem pressa
8 . A História do Cinema para quem tem pressa
9 . A História de Jesus para quem tem pressa
10 . A HISTORIA DO BRASIL PARA QUEM TEM PRESSA
11 . História revelada da SS
12 . A ciência por trás do segredo
13 . Anos 80 de a - Z
14 . Criança tem cada uma
15 . O resto da história
16 . A Woman in Berlin
17 . Travels
18 . Ufologia: Uma CiÊncia para Conhecer e Difundir
19 . Coach
20 . The Survivor
21 . Morri para viver
22 . Hamlet
23 . A Moderna Ciência do Yoga
24 . Neruda
25 . Neruda
26 . Neruda
27 . Arrase!
28 . Quem tem fé não precisa de sorte
29 . Atonement


Objetivo: O sistema quer recomendar livros parecidos com um que você já gosta.

Como funciona:

Coleta de Dados: O sistema usa um conjunto de dados de livros com informações como título, autor, categorias e idioma.

Entendendo os Livros (Extração de Recursos): O sistema pega as informações mais importantes dos livros, como o título, autor e categorias. Imagine que ele cria uma "carteirinha de identidade" para cada livro com esses dados.

Criando o Perfil do Livro (Vetorização TF-IDF): O sistema transforma as informações da "carteirinha de identidade" em um código numérico especial (vetor). É como se cada livro ganhasse um código secreto que representa suas características. Um código que dá mais peso às palavras importantes e menos às comuns, usando TF-IDF.

Encontrando Livros Parecidos (Semelhança de Cosseno): O sistema compara os códigos secretos dos livros. Se dois livros têm códigos parecidos, significa que eles são semelhantes. Para isso ele usa a "Semelhança de Cosseno" que calcula o quão parecidos os códigos são.

Sua Escolha: Você diz ao sistema qual livro você gosta.

Busca por Livros Parecidos: O sistema encontra o código secreto do livro que você escolheu e procura outros livros com códigos parecidos.

Recomendação: O sistema te mostra os livros com os códigos mais parecidos, ou seja, os livros mais semelhantes ao que você gosta.

Em resumo: O sistema analisa as características dos livros, cria um código para cada um e compara os códigos para te recomendar livros parecidos com o que você escolheu.